In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

df1 = pd.read_csv('/content/drive/MyDrive/Datathons/HACIK2024/anticipating latency/training_data.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Datathons/HACIK2024/anticipating latency/testing_data.csv')
train = df1.copy()
test = df2.copy()
label_encoder = LabelEncoder()
train['cpu_type'] = label_encoder.fit_transform(train['cpu_type'])
test['cpu_type'] = label_encoder.transform(test['cpu_type'])

train['timestamp'] = pd.to_datetime(train['timestamp'])
train['hour'] = train['timestamp'].dt.hour
train['day'] = train['timestamp'].dt.day
train['month'] = train['timestamp'].dt.month
train.drop('timestamp', axis=1, inplace=True)

test['timestamp'] = pd.to_datetime(test['timestamp'])
test['hour'] = test['timestamp'].dt.hour
test['day'] = test['timestamp'].dt.day
test['month'] = test['timestamp'].dt.month
test.drop('timestamp', axis=1, inplace=True)
# Function to convert ram_limit from megabytes to bytes
def convert_ram_limit_to_bytes(ram_limit):
    return int(ram_limit.replace('M', '')) * (2 ** 20)

# Apply the function to the ram_limit column
train['ram_limit'] = train['ram_limit'].apply(convert_ram_limit_to_bytes)
test['ram_limit'] = test['ram_limit'].apply(convert_ram_limit_to_bytes)
X_train = train.drop(['ID','latency'], axis=1)
X_test = test.drop('ID', axis=1)
y_train = train['latency']
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(X_test)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

models = {
    'RandomForestRegressor': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(objective='reg:squarederror', random_state=42),
    'ExtraTreesRegressor': ExtraTreesRegressor(),
    'LGBMRegressor': LGBMRegressor(verbose=0),
    'CatBoostRegressor': CatBoostRegressor(verbose=0),
}

# Cross-validate each model and print R-squared scores
for name, model in models.items():
    scores = cross_val_score(model, train_scaled, y_train, cv=5, scoring='r2')
    print(f'{name}: R-squared = {np.mean(scores):.4f} (mean), {np.std(scores):.4f} (std)')


KeyboardInterrupt: 

In [ ]:
ExtraTreesRegressor_params = {'n_estimators': 155, 'max_depth': 36, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.7674469065319266}
CatBoost_params = {'iterations': 944, 'depth': 9, 'learning_rate': 0.04971279843705733, 'random_strength': 5, 'bagging_temperature': 0.1299944122871062, 'border_count': 204,
                   'l2_leaf_reg': 0.007338808750734919, 'verbose':0}
randomforest_params = {'n_estimators':400,'max_depth':13,'min_samples_split':5, 'min_samples_leaf':1, 'max_features':'auto','bootstrap':True}
xg_params = {'colsample_betree': 0.8677158030594335, 'learning_rate':0.08956071030451762,'max_depth':8,'min_child_weight':6,'n_estimators':279,'subsample':0.8930977246667604}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor, StackingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
import warnings

# Disable all warnings
warnings.filterwarnings('ignore')


# Define the base models
et_model = ExtraTreesRegressor(**ExtraTreesRegressor_params)
cb_model = CatBoostRegressor(**CatBoost_params)
rf_model = RandomForestRegressor(**randomforest_params)
xg_model = XGBRegressor(**xg_params)

# Define the voting regressor
voting_regressor1 = VotingRegressor(estimators=[
    ('xgboost', xg_model),
    ('rn', cb_model),
    ('extra', et_model)
])
# voting_regressor.fit(train_scaled, y_train)
# y_pred = voting_regressor.predict(test_scaled)

# sub = pd.DataFrame({'ID': df2['ID'], 'latency': y_pred})
# sub.to_csv('votingxgrnet.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor, StackingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
import warnings

# Disable all warnings
warnings.filterwarnings('ignore')


# Define the base models
et_model = ExtraTreesRegressor(**ExtraTreesRegressor_params)
cb_model = CatBoostRegressor(**CatBoost_params)
rf_model = RandomForestRegressor(**randomforest_params)
xg_model = XGBRegressor(**xg_params)

# Define the voting regressor
voting_regressor = VotingRegressor(estimators=[
    ('xgboost', xg_model),
    ('extra', et_model),
    ('random_forest', rf_model),
    ('catboost', cb_model)
])
voting_regressor.fit(train_scaled, y_train)
y_pred = voting_regressor.predict(test_scaled)

sub = pd.DataFrame({'ID': df2['ID'], 'latency': y_pred})
sub.to_csv('votingall0.778.csv', index=False)

# # Evaluate with cross-validation
# voting_scores = cross_val_score(voting_regressor, train_scaled, y_train, cv=5, scoring='r2')
# print(f'Voting Regressor: R-squared = {np.mean(voting_scores):.4f} (mean), {np.std(voting_scores):.4f} (std)')


In [ ]:
from sklearn.linear_model import Ridge
# Define the final estimator for stacking
final_estimator = Ridge()

# Define the base models
et_model = ExtraTreesRegressor(**ExtraTreesRegressor_params)
cb_model = CatBoostRegressor(**CatBoost_params)
rf_model = RandomForestRegressor(**randomforest_params)
xg_model = XGBRegressor(**xg_params)

# Define the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ('extra_trees', et_model),
        ('xgb', voting_regressor1),
        ('random_forest', cb_model)
    ],
    final_estimator=final_estimator,
    cv=5
)

stacking_regressor.fit(train_scaled, y_train)
y_pred = stacking_regressor.predict(test_scaled)

sub = pd.DataFrame({'ID': df2['ID'], 'latency': y_pred})
sub.to_csv('stacking.csv', index=False)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
best_model = ExtraTreesRegressor(**ExtraTreesRegressor_params)
best_model.fit(train_scaled, y_train)
y_pred = best_model.predict(test_scaled)

sub = pd.DataFrame({'ID': test['ID'], 'latency': y_pred})
sub.to_csv('submission.csv', index=False)

In [ ]:
# Evaluate with cross-validation
from sklearn.linear_model import Ridge
# Define the final estimator for stacking
final_estimator = Ridge()

# Define the base models
et_model = ExtraTreesRegressor(**ExtraTreesRegressor_params)
cb_model = CatBoostRegressor(**CatBoost_params)
rf_model = RandomForestRegressor(**randomforest_params)
xg_model = XGBRegressor(**xg_params)

# Define the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ('extra_trees', et_model),
        ('xgb', rf_model),
        ('random_forest', cb_model),
        ('xg', xg_model)
    ],
    final_estimator=final_estimator,
    cv=5
)

voting_scores = cross_val_score(stacking_regressor, train_scaled, y_train, cv=5, scoring='r2')
 print(f'Voting Regressor: R-squared = {np.mean(voting_scores):.4f} (mean), {np.std(voting_scores):.4f} (std)')


Voting Regressor: R-squared = 0.9776 (mean), 0.0037 (std)


In [ ]:
stacking_regressor.fit(train_scaled, y_train)
y_pred = stacking_regressor.predict(test_scaled)

sub = pd.DataFrame({'ID': test['ID'], 'latency': y_pred})
sub.to_csv('stacking0.9776.csv', index=False)